In [6]:
import cv2
import numpy as np
from math import acos, pi, sqrt, cos, e

In [7]:
#Mean, Gaussian, Median Filter 모두 같은 마스크 크기 적용
#사각형이라고 가정, 보통은 정사각형이라 3*3, 5*5, 7*7을 기준으로 생각할 것
maskheight = int(input("Height of square : "))
maskwidth = int(input("Width of square : "))

Height of square : 3
Width of square : 3


In [8]:
#방법1 hsi 사용하지 않고 바로 Channel값에 넣기 (Mean , Gaussian, Medium Filter 모두)

In [9]:
#위에서 지정한 크기의 마스크로
def meanfilter(src, height, width, maskheight, maskwidth):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #meanfilter는 mask라 사이즈 지정 잘 해주기
    mean = np.zeros((maskheight, maskwidth), dtype=np.float32)
    
    pixel_num = maskheight*maskwidth
    
    #mean filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            mean[i][j] = 1/pixel_num
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp += mean[m][n] * src[i+m][j+n]
            plt[i][j] = temp
            
    return plt

In [10]:
#위에서 지정한 크기의 마스크로
def gaussianfilter(src, height, width, maskheight, maskwidth):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #gausianfilter는 mask라 사이즈 지정 잘 해주기
    gaussian = np.zeros((maskheight, maskwidth), dtype=np.float32)
    
    #sigma 입력받기. sigma가 커질 수록 분포가 넓어짐
    sigma = int(input("가우시안 필터의 시그마 값을 정수로 입력해주세요 : "))

    g_sum = 0
    #gaussian filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
    for i in range(maskheight):
        for j in range(maskwidth):
            gaussian[i][j] = (1/(2*pi*sigma*sigma)) * e ** (-((i**2) + (j**2))/(sigma*sigma))
            g_sum += gaussian[i][j]
    
    for i in range(height):
        for j in range(width):
            if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                plt[i][j] = src[i][j]
                continue
            
            temp = 0
            for m in range(maskheight):
                for n in range(maskwidth):
                    temp += 1/g_sum * gaussian[m][n] * src[i+m][j+n]
            plt[i][j] = temp
            
    return plt

In [11]:
#위에서 지정한 크기의 마스크로

def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    lesser_arr, equal_arr, greater_arr = [], [], []
    for num in arr:
        if num < pivot:
            lesser_arr.append(num)
        elif num > pivot:
            greater_arr.append(num)
        else:
            equal_arr.append(num)
    return quick_sort(lesser_arr) + equal_arr + quick_sort(greater_arr)


def medianfilter(src, height, width, maskheight, maskwidth):
    #결과값
    plt = np.zeros((height, width,3), dtype=np.uint8)

    #medianfilter는 mask 값을 따로 가지지 않음. 이동하면 값이 바로바로 변함
    shape = int(input("필터의 모양을 선택해주세요. [1: 가로(너비만큼만), 2: 세로(높이만큼만), 3: 십자가모양(너비=높이이고 둘 다 홀수일때 가능),\n4: 사각형(너비*높이 형태), 5: X자형태(너비=높이이고 둘 다 홀수일때 가능), 6: 다이아몬드모양(너비=높이이고 둘 다 홀수일때 가능)]\n원하는 선택지 : "))
    median_B = []
    median_G = []
    median_R = []
    
    # Horizontal
    if shape == 1:
        #median[0:maskwidth] = []
        #median filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
        for i in range(height):
            for j in range(width):
                if j < maskwidth or j > width-maskwidth :
                    plt[i][j] = src[i][j]
                    continue
            
                median_B, median_G, median_R = [], [], []
                for m in range(1):
                    for n in range(maskwidth):
                        median_B.append(src[i+m][j+n][0])
                        median_G.append(src[i+m][j+n][1])
                        median_R.append(src[i+m][j+n][2])
            
                #크기순으로 정렬 (퀵소트로 해서 시간복잡도는 O(NlogN)으로 예상)
                median_B, median_G, median_R = quick_sort(median_B), quick_sort(median_G), quick_sort(median_R)
                plt[i][j][0] = median_B[maskwidth//2]
                plt[i][j][1] = median_G[maskwidth//2]
                plt[i][j][2] = median_R[maskwidth//2]
            
    # Vertical
    elif shape == 2:
        #median[0:maskheight] = [0,]
        #median filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
        for i in range(height):
            for j in range(width):
                if i < maskheight or i > height-maskheight :
                    plt[i][j] = src[i][j]
                    continue
            
                median_B, median_G, median_R = [], [], []
                for m in range(maskheight):
                    for n in range(1):
                        median_B.append(src[i+m][j+n][0])
                        median_G.append(src[i+m][j+n][1])
                        median_R.append(src[i+m][j+n][2])
            
                #크기순으로 정렬 (퀵소트로 해서 시간복잡도는 O(NlogN)으로 예상)
                median_B, median_G, median_R = quick_sort(median_B), quick_sort(median_G), quick_sort(median_R)
                plt[i][j][0] = median_B[maskwidth//2]
                plt[i][j][1] = median_G[maskwidth//2]
                plt[i][j][2] = median_R[maskwidth//2]
            
    # Cross
    elif shape == 3:
        #median[0:maskwidth+maskheight-1] = [0,]
        #median filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
        for i in range(height):
            for j in range(width):
                if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                    plt[i][j] = src[i][j]
                    continue
            
                median_B, median_G, median_R = [], [], []
                for m in range(maskheight):
                    for n in range(maskwidth):
                        if m != (maskheight - 1)//2 :     # 중간 행이 아닐 때
                            if n != (maskwidth - 1)//2 :  # 중간 열이 아니라면 건너뛰기
                                continue                  # 중간 행인 경우에는 다 append 해줌
                                
                        median_B.append(src[i+m][j+n][0])
                        median_G.append(src[i+m][j+n][1])
                        median_R.append(src[i+m][j+n][2])
            
                #크기순으로 정렬 (퀵소트로 해서 시간복잡도는 O(NlogN)으로 예상)
                median_B, median_G, median_R = quick_sort(median_B), quick_sort(median_G), quick_sort(median_R)
                plt[i][j][0] = median_B[(maskwidth+maskheight-1)//2]
                plt[i][j][1] = median_G[(maskwidth+maskheight-1)//2]
                plt[i][j][2] = median_R[(maskwidth+maskheight-1)//2]

    elif shape == 4:
        median[0:maskwidth*maskheight] = [0,]
        #median filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
        for i in range(height):
            for j in range(width):
                if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                    plt[i][j] = src[i][j]
                    continue
            
                median_B, median_G, median_R = [], [], []
                for m in range(maskheight):
                    for n in range(maskwidth):
                        median_B.append(src[i+m][j+n][0])
                        median_G.append(src[i+m][j+n][1])
                        median_R.append(src[i+m][j+n][2])
            
                #크기순으로 정렬 (퀵소트로 해서 시간복잡도는 O(NlogN)으로 예상)
                median_B, median_G, median_R = quick_sort(median_B), quick_sort(median_G), quick_sort(median_R)
                plt[i][j][0] = median_B[maskwidth//2]
                plt[i][j][1] = median_G[maskwidth//2]
                plt[i][j][2] = median_R[maskwidth//2]

    elif shape == 5:
        #median[0:maskwidth+maskheight-1] = [0,]
        #median filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
        for i in range(height):
            for j in range(width):
                if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                    plt[i][j] = src[i][j]
                    continue
            
                median_B, median_G, median_R = [], [], []
                for m in range(maskheight):
                    for n in range(maskwidth):
                        if n == m or n == maskwidth-1 - m :  # X자 모양이라
                            median_B.append(src[i+m][j+n][0])
                            median_G.append(src[i+m][j+n][1])
                            median_R.append(src[i+m][j+n][2])
            
                #크기순으로 정렬 (퀵소트로 해서 시간복잡도는 O(NlogN)으로 예상)
                median_B, median_G, median_R = quick_sort(median_B), quick_sort(median_G), quick_sort(median_R)
                plt[i][j][0] = median_B[(maskwidth+maskheight-1)//2]
                plt[i][j][1] = median_G[(maskwidth+maskheight-1)//2]
                plt[i][j][2] = median_R[(maskwidth+maskheight-1)//2]
            
    elif shape == 6:
        #median[0:maskwidth*maskheight - 4*(maskwidth-2)] = [0,]
        #median filter 중심으로 볼 좌표는 따로 정하지 않음. 보통 가운데 지점이 origin이 됨.
        for i in range(height):
            for j in range(width):
                if i < maskheight or i > height-maskheight or j < maskwidth or j > width-maskwidth :
                    plt[i][j] = src[i][j]
                    continue
            
                median_B, median_G, median_R = [], [], []
                temp = (maskwidth-1)//2     # maskheight의 딱 중간 지점
                for m in range(maskheight):
                    for n in range(maskwidth):
                        if (n <= temp and n >= maskwidth-temp-1) or (n >= temp and n <= maskwidth-temp-1) :
                            median_B.append(src[i+m][j+n][0])
                            median_G.append(src[i+m][j+n][1])
                            median_R.append(src[i+m][j+n][2])
                    
                    # m이 maskheight 중간 지점을 넘어가기 이전에는 masking 되는 부분을 2개씩 늘어나게 하기
                    if m < (maskheight+1)//2 :   
                        temp += 1
                        
                    # m이 maskheight 중간 지점을 넘어가면 masking 되는 부분을 2개씩 줄어들게 하기
                    else :    # m >= (maskwidth+1)//2
                        temp -= 1
                        
            
                #크기순으로 정렬 (퀵소트로 해서 시간복잡도는 O(NlogN)으로 예상)
                median_B, median_G, median_R = quick_sort(median_B), quick_sort(median_G), quick_sort(median_R)
                plt[i][j][0] = median_B[(maskwidth*maskheight - 4*(maskwidth-2))//2]
                plt[i][j][1] = median_G[(maskwidth*maskheight - 4*(maskwidth-2))//2]
                plt[i][j][2] = median_R[(maskwidth*maskheight - 4*(maskwidth-2))//2]
            
    
    
    return plt

In [15]:
# B,G,R
def compute_Hue(B, G, R):
    # prevent divide by 0, adding 0.00000001
    angle = 0.5*((R - G) + (R - B)) / sqrt((R-G)*(R-G) + (R-B)*(G-B) + 0.00000001)
    if B<= G:
        return acos(angle)
    elif B> G:
        return (2*pi- acos(angle))
    #return acos(angle) if B <= G else (2*pi - acos(angle))

In [ ]:
src = cv2.imread('./Salt&pepper noise.png', cv2.IMREAD_COLOR)
height, width = src.shape[0], src.shape[1]
        
mean = np.zeros((height, width,3), dtype=np.uint8)
gaussian = np.zeros((height, width,3), dtype=np.uint8)
median = np.zeros((height, width,3), dtype=np.uint8)
        
mean = meanfilter(src, height, width, maskheight, maskwidth)
gaussian = gaussianfilter(src, height, width, maskheight, maskwidth)
'''
for i in range(6):
    median = medianfilter(src, height, width, maskheight, maskwidth)
    cv2.imshow("Median Filter "+str(i+1), median)
'''
cv2.imshow("Mean Filter", mean)
cv2.imshow("Gaussian Filter", gaussian)
cv2.imshow("src", src)
cv2.waitKey(0)
cv2.destroyAllWindows()

가우시안 필터의 시그마 값을 정수로 입력해주세요 : 2
